In [29]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import json

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

/home/ubuntu/real_estate_dashboard


In [1]:
file_dir = '0_data/geoservice/sig.shp'
geo_data = gpd.read_file(file_dir, dtype={'SIG_CD':object}, encoding='utf-8')
geo_data.head()

In [31]:
geo_data.crs = 'EPSG:5179'
geo_data = geo_data.to_crs({"init": "EPSG:4326"})

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [2]:
geo_data['SIG_KOR_NM'] = geo_data['SIG_KOR_NM'].str.decode('euc-kr')
geo_data.head()

In [3]:
# 법정동코드 로드
logal_info_b = pd.read_csv('0_data/legal_info_b.csv', dtype = {'법정동코드':str})

# 시도코드 생성
logal_info_b['시도코드'] = logal_info_b['법정동코드'].str[:2]

# 시도 코드, 시도명 코드 생성
logal_info_b_2 = logal_info_b[['시도코드', '시도명']].drop_duplicates().reset_index(drop = True)
logal_info_b_2 


In [36]:
geo_data['시도코드'] = geo_data['SIG_CD'].str[:2]

In [4]:
geo_data = pd.merge(geo_data,logal_info_b_2,
        on = '시도코드', 
        how = 'left')
geo_data.head()

In [5]:
# json 파일로 저장
geo_data.to_file('0_data/geoservice/geo_sig_json.geojson', encoding = 'utf-8', driver = 'GeoJSON')

In [52]:
for i in range(len(logal_info_b_2)):
    sd_code = logal_info_b_2['시도코드'][i]
    sd_nm = logal_info_b_2['시도명'][i]
    
    geo_data_cd = geo_data[geo_data['시도코드'] == sd_code]
    
    geo_data_cd.to_file(f'0_data/geoservice/geo_sig_{sd_nm}_json.geojson', encoding = 'utf-8', driver = 'GeoJSON')